In [0]:
#Univariate Hist of all float_64, and int64 or any desired dtypes and also includes the option of removing certain dtypes
def uni_con_dist(data, to_leave):
  data.drop(to_leave, axis = 1, inplace = True)
  data[data.dtypes[((data.dtypes=="float64")|(data.dtypes=="int64"))].index.values].hist(figsize=[11,11])
  
  hist_data = {}
  for col in data[data.dtypes[((data.dtypes=="float64")|(data.dtypes=="int64"))].index.values].columns.values:
    hist_data[col + '_count'], hist_data[col + '_division'] = np.histogram(data[col])
  
  return hist_data
  
#While pass dataframe to uni_con_dist we always pass a copy of dataframe. i.e. df.copy(deep = False)

In [0]:
#Biavariate Con-Con
def bi_con_con(data, to_leave, size = 10):
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt

    data.drop(to_leave, axis = 1, inplace = True)

    sns.pairplot(data.loc[:, data.dtypes[((data.dtypes=="float64")|(data.dtypes=="int64"))].index.values], size = size)
    data_corr = data.loc[:, data.dtypes[((data.dtypes=="float64")|(data.dtypes=="int64"))].index.values].corr()
    return data_corr

In [0]:
#Biavariate Cat-Cat Two Way table
def bi_cat_cat_2way(label1, label2, data):
  two_way = {}
  two_way['normal'] = pd.crosstab(index=data[label1], columns=data[label2], margins=True)# Usual 2 way table b/w 2 categorical vars
  two_way['total_prop'] = two_way['normal']/two_way['normal'].loc["All","All"] #to get the total proportion of counts in each cell,
                                                                               #divide the table by the grand total
  two_way['col_prop'] = two_way['normal']/two_way['normal'].loc["All", :]
  two_way['row_prop'] = (two_way['normal'].T/two_way['normal']['All']).T
  return two_way

In [0]:
#Bivariate Cat-Cat Chi Square and Phi Coefficient #Used when 2 Way table = 2 X 2
def bi_cat_cat_phi(two_way_normal):
  observed = two_way_normal.iloc[0:-1,0:-1]
  chi_stat, p_val, df, expected = scipy.stats.chi2_contingency(observed)
  phi = np.sqrt(chi_stat/two_way_normal['All'].iloc[-1])
  for name in ['chi_stat','p_val','df','expected', 'phi']:
    chi2_contingency[name] = eval(name)
  return chi2_contingency

In [0]:
#Bivariate Cat-Cat Chi Square and Cramers'V #Used when 2 Way table > 2 X 2
def bi_cat_cat_cramerV(two_way_normal):
  observed = two_way_normal.iloc[0:-1,0:-1]
  chi_stat, p_val, df, expected = scipy.stats.chi2_contingency(observed)
  cramers_v = np.sqrt(chi_stat/(two_way_normal['All'].iloc[-1] *(min(observed.shape)-1)))
  chi2_contingency = {}
  for name in ['chi_stat','p_val','df','expected', 'cramers_v']:
    chi2_contingency[name] = eval(name)
  #chi2_contingency = dict((name,eval(name)) for name in ['chi_stat','p_val','df','expected'])
  #chi2_contingency['cramers_v'] = cramers_v
  return chi2_contingency

In [0]:
#Bivariate Cat-Cat Chi Square and Contingency Coefficient
def bi_cat_cat_contincoeff(two_way_normal):
  observed = two_way_normal.iloc[0:-1,0:-1]
  chi_stat, p_val, df, expected = scipy.stats.chi2_contingency(observed)
  cc = np.sqrt(chi_stat/(two_way_normal['All'].iloc[-1] + chi_stat))
  for name in ['chi_stat','p_val','df','expected', 'cc']:
    chi2_contingency[name] = eval(name)
  return chi2_contingency

In [0]:
#Bivariate Cat-Con ANOVA
def bi_cat_con_anova(cat_label, con_label, data):
  features = {}
  features['grand_mean'] = 0
  features['I'] = len(data[cat_label].value_counts())
  features['total_samples'] = np.sum(data[cat_label].value_counts().values.tolist())
  
  i = 1
  
  for categ in data[cat_label].value_counts().index.tolist():
    features['g' + str(i)] = data.loc[data[cat_label] == categ, con_label]
    features['n' + str(i)] = len(features['g' + str(i)])
    features['x' + str(i)] = features['g' + str(i)].mean()
    features['grand_mean']+= features['x' + str(i)]/features['I']
    
    i+=1
    if i > features['I']:
      break
    
    
  features['sb2'] = np.sum(features['n' + str(i)] * np.power(features['x' + str(i)] - features['grand_mean'], 2) for i in range(1, features['I'] + 1))/(features['I']-1) 
  features['sw2'] = np.sum((features['n' + str(i)] - 1) * np.var(features['g' + str(i)], ddof = 1) for i in range(1, features['I'] + 1))/(features['total_samples'] - features['I'])
  features['f_stat'] = features['sb2']/features['sw2']
  features['df_num'] = features['I'] - 1
  features['df_denom'] = features['total_samples'] - features['I']
    
  
  return features